In [1]:
%load_ext Cython

In [2]:
import json
from datetime import datetime
import optuna as op
from optuna.samplers import TPESampler
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Utils.recsys2022DataReader import createCombineURM_ICMAll

In [5]:
URM2 = createCombineURM_ICMAll()
URM2

/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/recsys2022DataReader.py:37: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(urmPath)


23091


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
URM_train_list = []
URM_validation_list = []

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM2, train_percentage=0.80)

for k in range(1):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage=0.80)
    URM_train_list.append(URM_train)
    URM_validation_list.append(URM_validation)

recommender_SLIMElasticNet_list = []

In [ ]:
def objective(trial):

    MAP_List_Fold = []

    alpha = trial.suggest_float("alpha", 0.00001, 0.1, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.000001, 0.1, log=True)
    topK = trial.suggest_int("topK", 300, 400, log=True)

    for index in range(len(URM_train_list)):
        recommender_SLIMElasticNet = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_list[index], verbose=False)

        recommender_SLIMElasticNet.fit(alpha=alpha, l1_ratio=l1_ratio, topK=topK)

        recommender_SLIMElasticNet_list.append(recommender_SLIMElasticNet)

        evaluator_validation = EvaluatorHoldout(URM_validation_list[index], cutoff_list=[10])

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender_SLIMElasticNet)

        MAP_List_Fold.append(result_dict.iloc[0]["MAP"])

    return sum(MAP_List_Fold) / len(MAP_List_Fold)

In [ ]:
study = op.create_study(direction='maximize', sampler=TPESampler(multivariate=True))
study.optimize(objective, n_trials=3)

In [ ]:
alpha = study.best_params['alpha']
l1_ratio = study.best_params['l1_ratio']
topK = study.best_params['topK']

In [ ]:
op.visualization.plot_optimization_history(study)

In [ ]:
op.visualization.plot_param_importances(study)

In [ ]:
recommender_SLIMElasticNet_list[0].fit(alpha=alpha, l1_ratio=l1_ratio, topK=topK)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(recommender_SLIMElasticNet_list[0])
result_dict.iloc[0]["MAP"]

In [ ]:
resultParameters = result_dict.to_json(orient="records")
parsed = json.loads(resultParameters)

In [ ]:
with open("logs/" + recommender_SLIMElasticNet_list[0].RECOMMENDER_NAME + "_logs_" + datetime.now().strftime('%b%d_%H-%M-%S') + ".json", 'w') as json_file:
    json.dump(study.best_params, json_file, indent=4)
    json.dump(parsed, json_file, indent=4)